## Data Loader

The following illustrates the process for writing timeseries data to InfluxDB.  Both Excel (Scintilometer and Davis Weather Station) and web request (NBDC, NOAA, and Sunrise/Sunset) data are included for overview purposes, enabling database updates at an arbitrary frequency.

The [InfluxDB](https://www.influxdata.com/) server is hosted over [port 8086](http://localhost:8086), and supports write, read, update, and delete operations through a UI or programatically with Python or developer tools.

Currently, the methods enable a user to process, and optionally to write records to a time-series database in the InfluxDB framework, data from the USNA scintillometer and Davis weather station.

The methods support four global flags, descrbed below:
* INFLUX_DB: Whether or not to write all records to an InfluxDB instance (requires url, token, org, and bucket names).
* RETURN_DF: Whether to return a Pandas DataFrame of records as method output.
* WRITE_CSV: Whether to write the processed, time standardized data to a CSV file in the data/in/.../processed directory (or arbitrary fp_out path).
* DEBUG: Whether to print file locations as they are read in memory.

#### Core Imports

In [451]:
import os
import datetime
import copy

import pandas as pd
import numpy as np
import glob

#### InfluxDB Imports

In [ ]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

#### Data Paths

In [438]:
BASE_DIR = os.path.abspath('..')
DATA_DIR = os.path.join(BASE_DIR, 'data')

SCINTILLOMETER_DIR = os.path.join(DATA_DIR, 'in', 'scintillometer')
SCINTILLOMETER_DIR_RAW = os.path.join(SCINTILLOMETER_DIR, 'raw')
SCINTILLOMETER_DIR_PROCESSED = os.path.join(SCINTILLOMETER_DIR, 'processed')

WEATHER_DIR = os.path.join(DATA_DIR, 'in', 'weather')
WEATHER_DIR_RAW = os.path.join(WEATHER_DIR, 'raw')
WEATHER_DIR_PROCESSED = os.path.join(WEATHER_DIR, 'processed')

NDBC_DIR =  os.path.join(DATA_DIR, 'in', 'ndbc')

In [439]:
print(f'Base Directory: {BASE_DIR}')
print('-----')
print(f'Scintillometer Directory: {SCINTILLOMETER_DIR}')
print(f'Raw Scintillometer Data: {SCINTILLOMETER_DIR_RAW}')
print(f'Processed Scintillometer Data: {SCINTILLOMETER_DIR_PROCESSED}')
print('-----')
print(f'Weather Station Directory: {WEATHER_DIR}')
print(f'Raw Weather Station Data: {WEATHER_DIR_RAW}')
print(f'Processed Weather Station Data: {WEATHER_DIR_PROCESSED}')
print('-----')
print(f'Base NDBC Data Directory: {NDBC_DIR}')

Base Directory: C:\Users\chris\Documents\GitHub\arg
-----
Scintillometer Directory: C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer
Raw Scintillometer Data: C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\raw
Processed Scintillometer Data: C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\processed
-----
Weather Station Directory: C:\Users\chris\Documents\GitHub\arg\data\in\weather
Raw Weather Station Data: C:\Users\chris\Documents\GitHub\arg\data\in\weather\raw
Processed Weather Station Data: C:\Users\chris\Documents\GitHub\arg\data\in\weather\processed
-----
Base NDBC Data Directory: C:\Users\chris\Documents\GitHub\arg\data\in\ndbc


#### (Optional) NDBC Data URLs (Useful for Water Temperature)

In [524]:
NDBC_URLS = [
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2c2019.txt.gz&dir=data/historical/cwind/', # TPLM2 2019 wind
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2c2020.txt.gz&dir=data/historical/cwind/', # TPLM2 2020 wind
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm212021.txt.gz&dir=data/cwind/Jan/', # TPLM2 Jan 2021 wind
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm222021.txt.gz&dir=data/cwind/Feb/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm222021.txt.gz&dir=data/cwind/Feb/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm232021.txt.gz&dir=data/cwind/Mar/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm242021.txt.gz&dir=data/cwind/Apr/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm252021.txt.gz&dir=data/cwind/May/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm262021.txt.gz&dir=data/cwind/Jun/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm272021.txt.gz&dir=data/cwind/Jul/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm282021.txt.gz&dir=data/cwind/Aug/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm292021.txt.gz&dir=data/cwind/Sep/', # TPLM2 Sep 2021 wind
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2h2019.txt.gz&dir=data/historical/stdmet/', # TPLM2 2019 met
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2h2020.txt.gz&dir=data/historical/stdmet/', # TPLM2 2020 met
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm212021.txt.gz&dir=data/stdmet/Jan/', # TPLM2 Jan 2021 met
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm222021.txt.gz&dir=data/stdmet/Feb/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm222021.txt.gz&dir=data/stdmet/Feb/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm232021.txt.gz&dir=data/stdmet/Mar/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm242021.txt.gz&dir=data/stdmet/Apr/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm252021.txt.gz&dir=data/stdmet/May/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm262021.txt.gz&dir=data/stdmet/Jun/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm272021.txt.gz&dir=data/stdmet/Jul/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm282021.txt.gz&dir=data/stdmet/Aug/',
    'https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm292021.txt.gz&dir=data/stdmet/Sep/' # TPLM2 Sep 2021 met
]

#### (Optional) InfluxDB values

In [539]:
url='http://localhost:8086'
token='8KDJOKSB-uhpDgfiY5sHTLpKLu-4o7vk9cFREQaJomzgUKzyqi26FDb-13tu844IiQeghsqS4Mjy2Or5Gcc3uw=='
org='USNA'
bucket='arg'

In [ ]:
client = influxdb_client.InfluxDBClient(
    url=url,
    token=token,
    org=org
)

# Use the Client Write API
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

In [527]:
INFLUX_DB_NAN = -999

#### Globals for data loading

In [262]:
INFLUX_DB = True
RETURN_DF = True
WRITE_CSV = True
DEBUG = True

DLST = {
    "2019": [datetime.datetime.fromisoformat('2019-03-10T02:00:00'), datetime.datetime.fromisoformat('2019-11-03T02:00:00'), datetime.timedelta(hours=1)],
    "2020": [datetime.datetime.fromisoformat('2020-03-08T02:00:00'), datetime.datetime.fromisoformat('2020-11-01T02:00:00'), datetime.timedelta(hours=1)],
    "2021": [datetime.datetime.fromisoformat('2021-03-14T02:00:00'), datetime.datetime.fromisoformat('2021-11-07T02:00:00'), datetime.timedelta(hours=1)],
    "2022": [datetime.datetime.fromisoformat('2022-03-13T02:00:00'), datetime.datetime.fromisoformat('2022-11-06T02:00:00'), datetime.timedelta(hours=1)],
}

In [175]:
def _add_daylight_savings(df: pd.DataFrame, dlst: dict = DLST) -> pd.DataFrame:
    """Adjusts time timstamps in the DataFrame to conform with DLST.

    Using the dictionary of supplied daylight savings start and end
    timestamps, this method adjusts records within the daylight
    savings interval such that all are in Eastern time, without 
    forcing a timezone on the column. Duplicate timestamps are
    removed with a preferance for the last time stamp.

    Args:
        df: The path to the scintillometer data directory containing raw
          data measurements.
        dlst: The dictinary of records for daylight savings start
          and end timestamps, along with the time adjustment of the form
          
          [timestamp_start, timestamp_end, hour_change]

    Returns:
        A modified Pandas DataFrame with adjustements made to timestamps.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.
    """
    for k, v in dlst.items():
        time_start = v[0]
        time_end = v[1]
        hour_change = v[2]    
        df.loc[((df.timestamp >= time_start) & (df.timestamp <= time_end)), 'timestamp'] = df.loc[((df.timestamp >= time_start) & (df.timestamp <= time_end)), 'timestamp'] + hour_change
    
    df = df.groupby('timestamp').last()
    return df.reset_index()


In [528]:
def write_to_influx(url: str, token: str, org: str, bucket: str, df: pd.DataFrame, table_name: str, influx_db_nan: int = INFLUX_DB_NAN) -> None:
    """
    write to Influx DB with syncronous batch API
    """
    if 'cn2' in [col.lower() for col in list(df.columns)]:
        fillna = 0 # replace with non-physical measurement as Influx fails with NaN
    else:
        fillna = influx_db_nan # replace with non-physical measurement as Influx fails with NaN
    
    with InfluxDBClient(url=url, token=token, org=org) as client:
        with client.write_api() as write_api:
            write_api.write(bucket=bucket, record=df.fillna(fillna),
                            data_frame_tag_columns=['tag'],
                            data_frame_measurement_name=table_name)


In [263]:
print(f'USING INFLUX DB FORMAT: {INFLUX_DB!s:^5} ')
print(f'RETURNING PANDAS DATAFRAME: {RETURN_DF!s:^5} ')
print(f'WRITTING DATA TO CSV: {WRITE_CSV!s:^5} ')
print(f'LOGGING DEBUG: {DEBUG!s:^5} ')

USING INFLUX DB FORMAT: True  
RETURNING PANDAS DATAFRAME: True  
WRITTING DATA TO CSV: True  
LOGGING DEBUG: True  


#### Scintilometer data loader

In [264]:
def _get_df_from_records_scintillometer(fp_in: str = SCINTILLOMETER_DIR_RAW, debug: bool = DEBUG):
    """Reads scintillometer records to Pandas DataFrame.

    Retrieves scintillometer records from Excel workbooks and reads them
    into a Pandas DataFrame, appending all records to a single DataFrame, 
    sorting the records by timestamp, and returning the DataFrame.

    Args:
        fp_in: The path to the scintillometer data directory containing raw
          data measurements.

    Returns:
        A Pandas DataFrame object with two columns, the timestamp and the 
          Cn2 reading produced by the scintillometer.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.

    Raises:
        IOError: An error occurred accessing any file in the supplied dir.
    """
    df_sc = None
    try:
        for name in glob.glob(str(os.path.join(fp_in, '*.xlsx'))):
            if debug:
                print(name)
            if df_sc is None:
                df_sc = pd.read_excel(name, parse_dates=[1], dtype={'Cn2': np.float64}, header=[8])
            else:
                df_sc = df_sc.append(pd.read_excel(name, parse_dates=[1], dtype={'Cn2': np.float64}, header=[8]))

        df_sc.sort_values(by='timestamp', inplace=True)
        return df_sc
    except:
        raise IOError


In [529]:
def read_scintillometer_from_file(url: str = None, token: str = None, org: str = None, bucket: str = None, fp_in: str = SCINTILLOMETER_DIR_RAW, fp_out: str = SCINTILLOMETER_DIR_PROCESSED, influx_db:bool = INFLUX_DB, return_df: bool = RETURN_DF, write_csv:bool = WRITE_CSV, debug: bool = DEBUG) -> pd.DataFrame:
    """Reads scintillometer records to Pandas DataFrame.

    Retrieves scintillometer records from Excel workbooks and reads them
    into a Pandas DataFrame, appending all records to a single DataFrame, 
    sorting the records by timestamp, and returning the DataFrame.

    Args:
        fp_in: The path to the scintillometer data directory containing raw
          data measurements.

    Returns:
        A Pandas DataFrame object with two columns, the timestamp and the 
          Cn2 reading produced by the scintillometer.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.

    Raises:
        IOError: An error occurred accessing any file in the supplied dir.
    """
    df_sc = _get_df_from_records_scintillometer(fp_in=fp_in, debug=debug)
    df_sc = _add_daylight_savings(df_sc)
            
    if write_csv:
        df_sc.to_csv(os.path.join(fp_out, 'scintillometer.csv'))
    
    if influx_db and all([url,token,org,bucket]):
        df_sc.drop(columns=['error'], inplace=True)
        df_sc['tag_location_name'] = 'usna'
        df_sc['tag_source'] = 'BLS450'
        df_sc['tag_location'] = '38.983N 76.479W'
        df_sc.set_index('timestamp', inplace=True)
        write_to_influx(url=url, token=token, org=org, bucket=bucket, df=df_sc, table_name='cn2', influx_db_nan=INFLUX_DB_NAN)
    
    if return_df:
        return df_sc
    else:
        del df_sc


#### Davis Weather Station data loader

In [530]:
def _get_df_from_records_weather(fp_in: str = WEATHER_DIR_RAW, debug: bool = DEBUG):
    """Reads Davis weather station records to Pandas DataFrame.

    Retrieves weather records from Excel workbooks and reads them
    into a Pandas DataFrame, appending all records to a single DataFrame, 
    sorting the records by timestamp, and returning the DataFrame.

    Args:
        fp_in: The path to the scintillometer data directory containing raw
          data measurements.

    Returns:
        A Pandas DataFrame object with two columns, the timestamp and the 
          Cn2 reading produced by the scintillometer.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.

    Raises:
        IOError: An error occurred accessing any file in the supplied dir.
    """
    df_weather = None
    try:
        for name in glob.glob(str(os.path.join(fp_in, '*.xlsx'))):
            if debug:
                print(name)
            if df_weather is None:
                df_weather = pd.read_excel(name, skiprows = range(2), parse_dates = [[0,1]], header = None, na_values={'---', '------'}, dtype={'2:7': np.float64, '8': str, '9:10': np.float64, '11': str, '12:37': np.float64}) #dtype={'Cn2': np.float64}
            else:
                df_weather = df_weather.append(pd.read_excel(name, skiprows = range(2), parse_dates = [[0,1]], header = None, na_values={'---', '------'}, dtype={'2:7': np.float64, '8': str, '9:10': np.float64, '11': str, '12:37': np.float64}))
        df_weather.columns = ['timestamp', 'temperature_air', 'temperature_air_high', 'temperature_air_low', 'humidity', 'dew_point_temperature', 'wind_speed', 'wind_direction', 'wind_run', 'wind_speed_high', 'wind_directrion_high', 'wind_chill', 'heat_index', 'thw_index', 'thsw_index', 'pressure', 'rain', 'rain_rate', 'solar_radiation', 'solar_energy', 'solar_radiation_high', 'uv_index', 'uv_dose', 'uv_high', 'heat_dd', 'cool_dd', 'inner_temperature', 'inner_humidity', 'inner_dew_point_temperature', 'inner_heat', 'inner_emc', 'air_density', 'evapotranspiration', 'wind_samp', 'wind_tx', 'iss_recept', 'arc_int']
        df_weather.sort_values(by='timestamp', inplace=True)
        return df_weather
    except:
        raise IOError


In [281]:
def read_weather_from_file(url: str = None, token: str = None, org: str = None, bucket: str = None, fp_in: str = WEATHER_DIR_RAW, fp_out: str = WEATHER_DIR_PROCESSED, influx_db:bool = INFLUX_DB, return_df: bool = RETURN_DF, write_csv:bool = WRITE_CSV, debug: bool = DEBUG) -> pd.DataFrame:
    """Reads weather records to Pandas DataFrame.

    Retrieves weather records from Excel workbooks and reads them
    into a Pandas DataFrame, appending all records to a single DataFrame, 
    sorting the records by timestamp, and returning the DataFrame.

    Args:
        fp_in: The path to the weather data directory containing raw
          data measurements.

    Returns:
        A Pandas DataFrame object with two columns, the timestamp and the 
          readings produced by the weatjer station.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.

    Raises:
        IOError: An error occurred accessing any file in the supplied dir.
    """
    df_weather = _get_df_from_records_weather(fp_in=fp_in, debug=debug)
    
    if write_csv:
        df_weather.to_csv(os.path.join(fp_out, 'weather.csv'))
    
    if influx_db and all([url,token,org,bucket]):
        df_weather['tag_location_name'] = 'usna'
        df_weather['tag_source'] = 'VANTAGE_PRO2'
        df_weather['tag_location'] = '38.983N 76.479W'
        df_weather.set_index('timestamp', inplace=True)
        write_to_influx(url=url, token=token, org=org, bucket=bucket, df=df_weather, table_name='davis_weather', influx_db_nan=INFLUX_DB_NAN)
    
    if return_df:
        return df_weather
    else:
        del df_weather


#### (Optional) NDBC Web Request Data (Useful for Water Temperature)

In [415]:
def _ndbc_stdmet_web_request_to_df(url: str, debug: bool = DEBUG) -> pd.DataFrame:
    response = requests.get(url)
    if response.status_code != 200: # web request did not succeed
        if debug:
            print(f'The web request to {url} was not successful, with code {response.status_code}.')
        return pd.DataFrame()

    rows = [row.split(' ') for row in response.text.split('\n')[2:]]
    row_tuples = []
    
    for row in rows:
        row_tuples.append(tuple([y for y in row if y != '']))
    
    df = pd.DataFrame.from_records(row_tuples, columns=['year', 'month', 'day', 'hour', 'minute', 'wind_direction', 'wind_speed', 'wind_speed_gust', 'wave_height', 'dpd', 'apd', 'mwd', 'pressure', 'temperature_air', 'temperature_water', 'dew_point_temperature', 'visibility', 'tide'])
    df.dropna(inplace=True)
    df.insert(0, 'timestamp', pd.to_datetime(df['day'].astype(str) \
                                 + '-' + df['month'].astype(str) \
                                 + '-' + df['year'].astype(str) \
                                 + ' ' + df['hour'].astype(str) \
                                 + ':' + df['minute'].astype(str), 
                                 format='%d-%m-%Y %H:%M'))
    
    df.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    
    df.wind_direction = pd.to_numeric(df.wind_direction)
    df.wind_speed = pd.to_numeric(df.wind_speed)
    df.wind_speed_gust = pd.to_numeric(df.wind_speed_gust)
    df.wave_height = pd.to_numeric(df.wave_height)
    df.dpd = pd.to_numeric(df.dpd)
    df.apd = pd.to_numeric(df.apd)
    df.mwd = pd.to_numeric(df.mwd)
    df.pressure = pd.to_numeric(df.pressure)
    df.temperature_air = pd.to_numeric(df.temperature_air)
    df.temperature_water = pd.to_numeric(df.temperature_water)
    df.dew_point_temperature = pd.to_numeric(df.dew_point_temperature)
    df.visibility = pd.to_numeric(df.visibility)
    df.tide = pd.to_numeric(df.tide)
    
    df = df.replace(999, np.nan)
    df = df.replace(99.0, np.nan)

    if debug:
        print(f'The web request to {url} was successful.')
        print(f'The returned DataFrame contains {len(df)} rows with columns {list(df.columns)}.')
    
    return df
    

In [502]:
def _ndbc_cwind_web_request_to_df(url: str, debug: bool = DEBUG) -> pd.DataFrame:
    response = requests.get(url)
    if response.status_code != 200: # web request did not succeed
        if debug:
            print(f'The web request to {url} was not successful, with code {response.status_code}.')
        return pd.DataFrame()
    rows = [row.split(' ') for row in response.text.split('\n')[2:]]
    row_tuples = []
    
    for row in rows:
        row_tuples.append(tuple([y for y in row if y != '']))
    
    df = pd.DataFrame.from_records(row_tuples, columns=['year', 'month', 'day', 'hour', 'minute', 'wind_direction', 'wind_speed', 'gdr', 'wind_speed_gust', 'gtime'])
    df.dropna(inplace=True)
    df.insert(0, 'timestamp', pd.to_datetime(df['day'].astype(str) \
                                 + '-' + df['month'].astype(str) \
                                 + '-' + df['year'].astype(str) \
                                 + ' ' + df['hour'].astype(str) \
                                 + ':' + df['minute'].astype(str), 
                                 format='%d-%m-%Y %H:%M'))
    
    df.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
    
    df.wind_direction = pd.to_numeric(df.wind_direction)
    df.wind_speed = pd.to_numeric(df.wind_speed)
    df.wind_speed_gust = pd.to_numeric(df.wind_speed_gust)
    df.gtime = pd.to_numeric(df.gtime)
    
    df = df.replace(9999, np.nan)
    df = df.replace(999, np.nan)
    df = df.replace(99.0, np.nan)
    
    if debug:
        print(f'The web request to {url} was successful.')
        print(f'The returned DataFrame contains {len(df)} rows with columns {list(df.columns)}.')

    return df

In [507]:
def _ndbc_web_request(url: str, data_request: str, debug: bool = DEBUG) -> pd.DataFrame:
    if data_request == 'stdmet':
        return _ndbc_stdmet_web_request_to_df(url=url, debug=debug)
    elif data_request == 'cwind':
        return _ndbc_cwind_web_request_to_df(url=url, debug=debug)
    else:
        raise NotImplementedError

In [531]:
def _get_staiton_lat_long(station_key: str ) -> str:
    try:
        request = requests.get(f'https://www.ndbc.noaa.gov/station_page.php?station={station_key}')
        lat_long = request.text.split(f'{station_key.upper()} (')[1].split(') - ')[0]
        return lat_long
    except:
        return 'unknown'

In [532]:
def read_ndbc_from_web(url: str = None, token: str = None, org: str = None, bucket: str = None, urls: list[str] = None, fp_out: str = NDBC_DIR, influx_db:bool = INFLUX_DB, return_df: bool = RETURN_DF, write_csv:bool = WRITE_CSV, debug: bool = DEBUG) -> dict:
    """Reads NDBC records to Pandas DataFrame.

    Retrieves NDBC records from Excel workbooks and reads them
    into a Pandas DataFrame, appending all records to a single DataFrame, 
    sorting the records by timestamp, and returning the DataFrame.

    Args: (TODO)
        url: The url of the web request to execute.

    Returns:
        A Pandas DataFrame object with two columns, the timestamp and the 
          readings produced by the weatjer station.

        The DataFrame is sorted by Timestamp, in monotonically increasing
          order.

    Raises:
        IOError: An error occurred accessing any file in the supplied dir.
    """
    dtypes = {}
    station_entry_format = {
        'urls': [],
        'date_start': 9999,
        'date_end': -1
    }
    
    for web_request in urls:
        request_args = web_request.split('https://www.ndbc.noaa.gov/view_text_file.php?filename=')[1]
        if 'cwind' in request_args:
            dtype = 'cwind'
        elif 'stdmet' in request_args:
            dtype = 'stdmet'
        else:
            raise NotImplementedError
        station_name = str(request_args[0:5])
        station_date = str(str(request_args[6:]).split('.')[0])
        
        if dtype not in dtypes.keys():
            dtypes[dtype] = {}
        if station_name not in dtypes[dtype].keys():
            dtypes[dtype].update({station_name: copy.deepcopy(station_entry_format)})
        dtypes[dtype][station_name]['urls'].append(web_request)
        dtypes[dtype][station_name]['date_start'] = min(dtypes[dtype][station_name]['date_start'], int(station_date))
        dtypes[dtype][station_name]['date_end'] = max(dtypes[dtype][station_name]['date_end'], int(station_date))
    
    ret = {}
    for dtype_key, dtype in dtypes.items():
        df_ndbc = None
        for station_key, station in dtype.items():
            for web_request in station['urls']:
                if df_ndbc is None:
                    df_ndbc = _ndbc_web_request(url=web_request, data_request=dtype_key, debug=debug)
                else:
                    df_ndbc.append( _ndbc_web_request(url=web_request, data_request=dtype_key, debug=debug))
                    
            _key = f'ndbc_{dtype_key}_{station_key}_{station["date_start"]}-{station["date_end"]}'
            ret[_key] = df_ndbc
            if write_csv:
                outdir = os.path.join(fp_out, dtype_key, station_key)
                outpath = os.path.join(outdir, f'{_key}.csv')
                if debug:
                    print(f'Writting data to {outpath}.')
                if not os.path.exists(outdir):
                    os.mkdir(outdir)
                df_ndbc.to_csv(outpath)
    
            if influx_db and all([url,token,org,bucket]):
                station_location_lat_long = _get_staiton_lat_long(station_key=station_key)
                
                df_ndbc['tag_location_name'] = 'ndbc_'
                df_ndbc['tag_source'] = f'NDBC_{station_key}'
                df_ndbc['tag_location'] = station_location_lat_long
                df_ndbc.set_index('timestamp', inplace=True)
                write_to_influx(url=url, token=token, org=org, bucket=bucket, df=df_ndbc, table_name=f'{station_key}_{dtype_key}')
    
    if return_df:
        return ret
    else:
        del ret

#### Process Scintillometer and Davis Weather Staion Data to InfluxDB and Local CSV

In [283]:
df_sc= read_scintillometer_from_file(url=url, token=token, org=org, bucket=bucket)

C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\raw\Cn2_Data_01JANUARY21_up_through_midday_24NOVEMBER21_remember_to_add_DST_as_appropriate.xlsx


Exception ignored in: <function InfluxDBClient.__del__ at 0x000001F75F4FD160>
Traceback (most recent call last):
  File "C:\Users\chris\miniconda3\envs\arg_pytorch\lib\site-packages\influxdb_client\client\influxdb_client.py", line 353, in __del__
    if self.api_client:
AttributeError: 'InfluxDBClient' object has no attribute 'api_client'


C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\raw\Data_and_error_messages_01JAN20_23JUN20.xlsx
C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\raw\Data_and_error_messages_14DEC20_31DEC20.xlsx
C:\Users\chris\Documents\GitHub\arg\data\in\scintillometer\raw\Data_and_error_messages_23JUN20_14DEC20.xlsx


2021-11-27 17:13:03,653 | Serialize chunk 1/979 ...
2021-11-27 17:13:03,682 | Serialize chunk 2/979 ...
2021-11-27 17:13:03,702 | Serialize chunk 3/979 ...
2021-11-27 17:13:03,730 | Serialize chunk 4/979 ...
2021-11-27 17:13:03,758 | Serialize chunk 5/979 ...
2021-11-27 17:13:03,786 | Serialize chunk 6/979 ...
2021-11-27 17:13:03,810 | Serialize chunk 7/979 ...
2021-11-27 17:13:03,832 | Serialize chunk 8/979 ...
2021-11-27 17:13:03,860 | Serialize chunk 9/979 ...
2021-11-27 17:13:03,881 | Serialize chunk 10/979 ...
2021-11-27 17:13:03,908 | Serialize chunk 11/979 ...
2021-11-27 17:13:03,930 | Serialize chunk 12/979 ...
2021-11-27 17:13:03,959 | Serialize chunk 13/979 ...
2021-11-27 17:13:03,986 | Serialize chunk 14/979 ...
2021-11-27 17:13:04,008 | Serialize chunk 15/979 ...
2021-11-27 17:13:04,030 | Serialize chunk 16/979 ...
2021-11-27 17:13:04,053 | Serialize chunk 17/979 ...
2021-11-27 17:13:04,075 | Serialize chunk 18/979 ...
2021-11-27 17:13:04,096 | Serialize chunk 19/979 ...
20

2021-11-27 17:13:07,803 | Serialize chunk 155/979 ...
2021-11-27 17:13:07,831 | Serialize chunk 156/979 ...
2021-11-27 17:13:07,855 | Serialize chunk 157/979 ...
2021-11-27 17:13:07,879 | Serialize chunk 158/979 ...
2021-11-27 17:13:07,902 | Serialize chunk 159/979 ...
2021-11-27 17:13:07,926 | Serialize chunk 160/979 ...
2021-11-27 17:13:07,948 | Serialize chunk 161/979 ...
2021-11-27 17:13:07,969 | Serialize chunk 162/979 ...
2021-11-27 17:13:07,991 | Serialize chunk 163/979 ...
2021-11-27 17:13:08,024 | Serialize chunk 164/979 ...
2021-11-27 17:13:08,046 | Serialize chunk 165/979 ...
2021-11-27 17:13:08,068 | Serialize chunk 166/979 ...
2021-11-27 17:13:08,092 | Serialize chunk 167/979 ...
2021-11-27 17:13:08,117 | Serialize chunk 168/979 ...
2021-11-27 17:13:08,138 | Serialize chunk 169/979 ...
2021-11-27 17:13:08,162 | Serialize chunk 170/979 ...
2021-11-27 17:13:08,185 | Serialize chunk 171/979 ...
2021-11-27 17:13:08,209 | Serialize chunk 172/979 ...
2021-11-27 17:13:08,231 | Se

2021-11-27 17:13:11,749 | Serialize chunk 307/979 ...
2021-11-27 17:13:11,784 | Serialize chunk 308/979 ...
2021-11-27 17:13:11,803 | Serialize chunk 309/979 ...
2021-11-27 17:13:11,811 | Serialize chunk 310/979 ...
2021-11-27 17:13:11,849 | Serialize chunk 311/979 ...
2021-11-27 17:13:11,884 | Serialize chunk 312/979 ...
2021-11-27 17:13:11,906 | Serialize chunk 313/979 ...
2021-11-27 17:13:11,930 | Serialize chunk 314/979 ...
2021-11-27 17:13:11,953 | Serialize chunk 315/979 ...
2021-11-27 17:13:11,979 | Serialize chunk 316/979 ...
2021-11-27 17:13:12,004 | Serialize chunk 317/979 ...
2021-11-27 17:13:12,027 | Serialize chunk 318/979 ...
2021-11-27 17:13:12,049 | Serialize chunk 319/979 ...
2021-11-27 17:13:12,074 | Serialize chunk 320/979 ...
2021-11-27 17:13:12,097 | Serialize chunk 321/979 ...
2021-11-27 17:13:12,130 | Serialize chunk 322/979 ...
2021-11-27 17:13:12,154 | Serialize chunk 323/979 ...
2021-11-27 17:13:12,179 | Serialize chunk 324/979 ...
2021-11-27 17:13:12,204 | Se

2021-11-27 17:13:15,729 | Serialize chunk 459/979 ...
2021-11-27 17:13:15,750 | Serialize chunk 460/979 ...
2021-11-27 17:13:15,790 | Serialize chunk 461/979 ...
2021-11-27 17:13:15,810 | Serialize chunk 462/979 ...
2021-11-27 17:13:15,839 | Serialize chunk 463/979 ...
2021-11-27 17:13:15,863 | Serialize chunk 464/979 ...
2021-11-27 17:13:15,888 | Serialize chunk 465/979 ...
2021-11-27 17:13:16,069 | Serialize chunk 466/979 ...
2021-11-27 17:13:16,092 | Serialize chunk 467/979 ...
2021-11-27 17:13:16,116 | Serialize chunk 468/979 ...
2021-11-27 17:13:16,137 | Serialize chunk 469/979 ...
2021-11-27 17:13:16,158 | Serialize chunk 470/979 ...
2021-11-27 17:13:16,182 | Serialize chunk 471/979 ...
2021-11-27 17:13:16,208 | Serialize chunk 472/979 ...
2021-11-27 17:13:16,229 | Serialize chunk 473/979 ...
2021-11-27 17:13:16,257 | Serialize chunk 474/979 ...
2021-11-27 17:13:16,289 | Serialize chunk 475/979 ...
2021-11-27 17:13:16,312 | Serialize chunk 476/979 ...
2021-11-27 17:13:16,335 | Se

2021-11-27 17:13:19,783 | Serialize chunk 611/979 ...
2021-11-27 17:13:19,807 | Serialize chunk 612/979 ...
2021-11-27 17:13:19,829 | Serialize chunk 613/979 ...
2021-11-27 17:13:19,853 | Serialize chunk 614/979 ...
2021-11-27 17:13:19,887 | Serialize chunk 615/979 ...
2021-11-27 17:13:19,912 | Serialize chunk 616/979 ...
2021-11-27 17:13:19,932 | Serialize chunk 617/979 ...
2021-11-27 17:13:19,958 | Serialize chunk 618/979 ...
2021-11-27 17:13:19,981 | Serialize chunk 619/979 ...
2021-11-27 17:13:20,005 | Serialize chunk 620/979 ...
2021-11-27 17:13:20,027 | Serialize chunk 621/979 ...
2021-11-27 17:13:20,049 | Serialize chunk 622/979 ...
2021-11-27 17:13:20,071 | Serialize chunk 623/979 ...
2021-11-27 17:13:20,091 | Serialize chunk 624/979 ...
2021-11-27 17:13:20,115 | Serialize chunk 625/979 ...
2021-11-27 17:13:20,145 | Serialize chunk 626/979 ...
2021-11-27 17:13:20,168 | Serialize chunk 627/979 ...
2021-11-27 17:13:20,192 | Serialize chunk 628/979 ...
2021-11-27 17:13:20,214 | Se

2021-11-27 17:13:23,857 | Serialize chunk 763/979 ...
2021-11-27 17:13:23,878 | Serialize chunk 764/979 ...
2021-11-27 17:13:23,906 | Serialize chunk 765/979 ...
2021-11-27 17:13:23,929 | Serialize chunk 766/979 ...
2021-11-27 17:13:23,950 | Serialize chunk 767/979 ...
2021-11-27 17:13:23,977 | Serialize chunk 768/979 ...
2021-11-27 17:13:23,999 | Serialize chunk 769/979 ...
2021-11-27 17:13:24,021 | Serialize chunk 770/979 ...
2021-11-27 17:13:24,051 | Serialize chunk 771/979 ...
2021-11-27 17:13:24,074 | Serialize chunk 772/979 ...
2021-11-27 17:13:24,097 | Serialize chunk 773/979 ...
2021-11-27 17:13:24,120 | Serialize chunk 774/979 ...
2021-11-27 17:13:24,141 | Serialize chunk 775/979 ...
2021-11-27 17:13:24,163 | Serialize chunk 776/979 ...
2021-11-27 17:13:24,185 | Serialize chunk 777/979 ...
2021-11-27 17:13:24,208 | Serialize chunk 778/979 ...
2021-11-27 17:13:24,232 | Serialize chunk 779/979 ...
2021-11-27 17:13:24,255 | Serialize chunk 780/979 ...
2021-11-27 17:13:24,276 | Se

2021-11-27 17:13:27,903 | Serialize chunk 915/979 ...
2021-11-27 17:13:27,925 | Serialize chunk 916/979 ...
2021-11-27 17:13:27,949 | Serialize chunk 917/979 ...
2021-11-27 17:13:27,979 | Serialize chunk 918/979 ...
2021-11-27 17:13:28,001 | Serialize chunk 919/979 ...
2021-11-27 17:13:28,025 | Serialize chunk 920/979 ...
2021-11-27 17:13:28,047 | Serialize chunk 921/979 ...
2021-11-27 17:13:28,068 | Serialize chunk 922/979 ...
2021-11-27 17:13:28,092 | Serialize chunk 923/979 ...
2021-11-27 17:13:28,116 | Serialize chunk 924/979 ...
2021-11-27 17:13:28,141 | Serialize chunk 925/979 ...
2021-11-27 17:13:28,162 | Serialize chunk 926/979 ...
2021-11-27 17:13:28,185 | Serialize chunk 927/979 ...
2021-11-27 17:13:28,205 | Serialize chunk 928/979 ...
2021-11-27 17:13:28,228 | Serialize chunk 929/979 ...
2021-11-27 17:13:28,250 | Serialize chunk 930/979 ...
2021-11-27 17:13:28,271 | Serialize chunk 931/979 ...
2021-11-27 17:13:28,296 | Serialize chunk 932/979 ...
2021-11-27 17:13:28,329 | Se

In [284]:
df_weather = read_weather_from_file(url=url, token=token, org=org, bucket=bucket)

C:\Users\chris\Documents\GitHub\arg\data\in\weather\raw\01JAN2021_to_24NOVEMBER_local_station_weather.xlsx
C:\Users\chris\Documents\GitHub\arg\data\in\weather\raw\01_JAN_to_31DECEMBER2020_local_station_weather.xlsx


2021-11-27 17:14:28,222 | Serialize chunk 1/95 ...
2021-11-27 17:14:28,419 | Serialize chunk 2/95 ...
2021-11-27 17:14:28,463 | Serialize chunk 3/95 ...
2021-11-27 17:14:28,507 | Serialize chunk 4/95 ...
2021-11-27 17:14:28,549 | Serialize chunk 5/95 ...
2021-11-27 17:14:28,588 | Serialize chunk 6/95 ...
2021-11-27 17:14:28,631 | Serialize chunk 7/95 ...
2021-11-27 17:14:28,675 | Serialize chunk 8/95 ...
2021-11-27 17:14:28,717 | Serialize chunk 9/95 ...
2021-11-27 17:14:28,758 | Serialize chunk 10/95 ...
2021-11-27 17:14:28,801 | Serialize chunk 11/95 ...
2021-11-27 17:14:28,850 | Serialize chunk 12/95 ...
2021-11-27 17:14:28,890 | Serialize chunk 13/95 ...
2021-11-27 17:14:28,935 | Serialize chunk 14/95 ...
2021-11-27 17:14:28,986 | Serialize chunk 15/95 ...
2021-11-27 17:14:29,031 | Serialize chunk 16/95 ...
2021-11-27 17:14:29,074 | Serialize chunk 17/95 ...
2021-11-27 17:14:29,124 | Serialize chunk 18/95 ...
2021-11-27 17:14:29,180 | Serialize chunk 19/95 ...
2021-11-27 17:14:29,2

#### Process NDBC Thomas Point Light Data to InfluxDB and Local CSV

In [540]:
df_ndbc_dict = read_ndbc_from_web(url=url, token=token, org=org, bucket=bucket, urls=NDBC_URLS, fp_out=NDBC_DIR, influx_db=INFLUX_DB, return_df=RETURN_DF, write_csv=WRITE_CSV, debug=DEBUG)

The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2c2019.txt.gz&dir=data/historical/cwind/ was successful.
The returned DataFrame contains 52441 rows with columns ['timestamp', 'wind_direction', 'wind_speed', 'gdr', 'wind_speed_gust', 'gtime'].
The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2c2020.txt.gz&dir=data/historical/cwind/ was successful.
The returned DataFrame contains 52609 rows with columns ['timestamp', 'wind_direction', 'wind_speed', 'gdr', 'wind_speed_gust', 'gtime'].
The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm212021.txt.gz&dir=data/cwind/Jan/ was successful.
The returned DataFrame contains 4464 rows with columns ['timestamp', 'wind_direction', 'wind_speed', 'gdr', 'wind_speed_gust', 'gtime'].
The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm222021.txt.gz&dir=data/cwind/Feb/ was successful.
The returned DataFrame contains 4020 rows with columns ['timestam

2021-11-28 15:09:22,728 | Serialize chunk 1/53 ...
2021-11-28 15:09:22,750 | Serialize chunk 2/53 ...
2021-11-28 15:09:22,774 | Serialize chunk 3/53 ...
2021-11-28 15:09:22,810 | Serialize chunk 4/53 ...
2021-11-28 15:09:22,844 | Serialize chunk 5/53 ...
2021-11-28 15:09:22,872 | Serialize chunk 6/53 ...
2021-11-28 15:09:22,909 | Serialize chunk 7/53 ...
2021-11-28 15:09:22,942 | Serialize chunk 8/53 ...
2021-11-28 15:09:22,980 | Serialize chunk 9/53 ...
2021-11-28 15:09:23,013 | Serialize chunk 10/53 ...
2021-11-28 15:09:23,051 | Serialize chunk 11/53 ...
2021-11-28 15:09:23,080 | Serialize chunk 12/53 ...
2021-11-28 15:09:23,104 | Serialize chunk 13/53 ...
2021-11-28 15:09:23,140 | Serialize chunk 14/53 ...
2021-11-28 15:09:23,171 | Serialize chunk 15/53 ...
2021-11-28 15:09:23,212 | Serialize chunk 16/53 ...
2021-11-28 15:09:23,252 | Serialize chunk 17/53 ...
2021-11-28 15:09:23,303 | Serialize chunk 18/53 ...
2021-11-28 15:09:23,354 | Serialize chunk 19/53 ...
2021-11-28 15:09:23,4

The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2h2019.txt.gz&dir=data/historical/stdmet/ was successful.
The returned DataFrame contains 8741 rows with columns ['timestamp', 'wind_direction', 'wind_speed', 'wind_speed_gust', 'wave_height', 'dpd', 'apd', 'mwd', 'pressure', 'temperature_air', 'temperature_water', 'dew_point_temperature', 'visibility', 'tide'].
The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm2h2020.txt.gz&dir=data/historical/stdmet/ was successful.
The returned DataFrame contains 8770 rows with columns ['timestamp', 'wind_direction', 'wind_speed', 'wind_speed_gust', 'wave_height', 'dpd', 'apd', 'mwd', 'pressure', 'temperature_air', 'temperature_water', 'dew_point_temperature', 'visibility', 'tide'].
The web request to https://www.ndbc.noaa.gov/view_text_file.php?filename=tplm212021.txt.gz&dir=data/stdmet/Jan/ was successful.
The returned DataFrame contains 744 rows with columns ['timestamp', 'wind_direction', 'win

2021-11-28 15:09:32,369 | Serialize chunk 1/9 ...
2021-11-28 15:09:32,397 | Serialize chunk 2/9 ...
2021-11-28 15:09:32,425 | Serialize chunk 3/9 ...
2021-11-28 15:09:32,453 | Serialize chunk 4/9 ...
2021-11-28 15:09:32,505 | Serialize chunk 5/9 ...
2021-11-28 15:09:32,538 | Serialize chunk 6/9 ...
2021-11-28 15:09:32,585 | Serialize chunk 7/9 ...
2021-11-28 15:09:32,668 | Serialize chunk 8/9 ...
2021-11-28 15:09:32,712 | Serialize chunk 9/9 ...


In [537]:
df_ndbc_dict.keys()

dict_keys(['ndbc_cwind_tplm2_2019-2021', 'ndbc_stdmet_tplm2_2019-2021'])

In [538]:
df_ndbc_cwind_tplm2_2019_2021 = df_ndbc_dict['ndbc_cwind_tplm2_2019-2021']
df_ndbc_cwind_tplm2_2019_2021 = df_ndbc_dict['ndbc_cwind_tplm2_2019-2021']